# Pandas demo
* Get functuion name DataFrame from test_items.xlsx using Pandas
* On Mac, Excel file is generated by free app "NUMBERS".


## Setup

```
% python --version
Python 3.9.0
% pip install pandas
% pip install openpyxl
% pip install jupyter
```

## 1. Check Excel file
* After creation by free app "NUMBERS", Excel file includes a sheet names "書き出しの概要".
* We should exclude this first sheet later.

In [95]:
import pandas as pd
import openpyxl
import glob

in_file = pd.ExcelFile("./data/test_items.xlsx")
print(in_file.sheet_names)


['書き出しの概要', 'Func1', 'Func2']


In [96]:
# show first sheet (auto generated by NUMBERS)
sheet0 = pd.read_excel("./data/test_items.xlsx", 0)
sheet0

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,NaN,この書類はNumbersから書き出されました。各表は Excelワークシート に変換されま...,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,Numbersシート名,Numbers表名,Excelワークシート名
6,NaN,NaN,NaN,NaN
7,NaN,Func1,NaN,NaN
8,NaN,NaN,表1,Func1
9,NaN,Func1,NaN,NaN


In [97]:
# sheet1 = pd.read_excel("./data/test_items.xlsx", sheet_name="Func1")
sheet1 = pd.read_excel("./data/test_items.xlsx", 1)
sheet1

,No.,分類1,分類2,テスト手順,期待結果,テスト関数
0,1,G1,G1-1,aaa,OK,test_parallel1
1,2,G1,G1-1,bbb,OK,test_serde1
2,3,G1,G1-2,ccc,OK,NaN
3,4,G1,G1-2,ddd,OK,test_serde1


In [98]:
# sheet2 = pd.read_excel("./data/test_items.xlsx", sheet_name="Func2")
sheet2 = pd.read_excel("./data/test_items.xlsx", 2)
sheet2

,No.,分類1,テスト手順,期待結果,テスト関数
0,1,group1,xxx,OK,test_parallel2
1,2,group1,yyy,OK,test_serde2
2,3,group1,zzz,NG,test_special2
3,4,group1,www,NG,test_another2


## 2. Get special raw from Excel file, column name "No.", "テスト関数"

In [104]:
out_data = []

for sheet_name in in_file.sheet_names:
    # exclude sheet made by numbers app.
    if sheet_name != '書き出しの概要':
        sheet = pd.read_excel("./data/test_items.xlsx", sheet_name)
        # print(sheet)
        for index, raw in sheet.iterrows():
            data = []
            data.append(sheet_name)
            data.append(raw["No."])
            data.append(raw["テスト関数"])
            # print(data)
            out_data.append(data)

print(out_data)

df = pd.DataFrame(out_data,
                  columns=['sheet_name', 'test_number', 'test_func'])
df


[['Func1', 1, 'test_parallel1'], ['Func1', 2, 'test_serde1'], ['Func1', 3, nan], ['Func1', 4, 'test_serde1'], ['Func2', 1, 'test_parallel2'], ['Func2', 2, 'test_serde2'], ['Func2', 3, 'test_special2'], ['Func2', 4, 'test_another2']]


,sheet_name,test_number,test_func
0,Func1,1,test_parallel1
1,Func1,2,test_serde1
2,Func1,3,NaN
3,Func1,4,test_serde1
4,Func2,1,test_parallel2
5,Func2,2,test_serde2
6,Func2,3,test_special2
7,Func2,4,test_another2


## 3. Select raws having same name of "test_func" column

In [100]:
test_serde1_lines = df[df["test_func"] == "test_serde1"]
test_serde1_lines

,sheet_name,test_number,test_func
1,Func1,2,test_serde1
3,Func1,4,test_serde1


In [101]:
# str.contains() can't get Nan.
# ---------------------------------------------------------------
# ValueError: cannot index with vector containing NA / NaN values
# ---------------------------------------------------------------
# So it must be set "na" value for Nan.
test_special2_lines = df[df["test_func"].str.contains("test_special2", na=False)]
test_special2_lines

,sheet_name,test_number,test_func
6,Func2,3,test_special2


In [102]:
# Pick up only NaN line.
test_nan_lines = df[df["test_func"].isnull()]
test_nan_lines

,sheet_name,test_number,test_func
2,Func1,3,NaN
